Mount Drive:

In [102]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


Set File Paths:

In [103]:
path_to_mrcnn = '/content/drive/MyDrive/AI/maskrcnn_files/Mask_RCNN/mrcnn/'
path_to_model = '/content/drive/MyDrive/AI/mask_rcnn_scar_finder_713map.h5'
path_to_all_images = '/content/drive/MyDrive/AI/data/all_images/'
path_to_save_annotations = '/content/drive/MyDrive/AI/data/predicted_annotations2/'
path_to_outline_poly = '/content/drive/MyDrive/AI/data/BLANK_SKETCH_updated.jpg.json'

Import Libraries:

In [104]:
import random
import json
from os import listdir
import numpy as np
import skimage
import time
from numpy import asarray
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from shapely.geometry import Polygon

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


In [105]:
import sys
sys.path.append(path_to_mrcnn)

from config import Config
import visualize
from utils import Dataset
import model as modellib
import utils
from model import log

Set Model Configurations:

In [106]:
class CustomConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """
    NAME = "scar_finder"      
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 3    # Background + Scar + Mute
    STEPS_PER_EPOCH = 100
    DETECTION_MIN_CONFIDENCE = 0.50    
    MAX_GT_INSTANCES = 120    
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)    
    LOSS_WEIGHTS = {'rpn_class_loss': 1.0,
                    'rpn_bbox_loss': 1.0,
                    'mrcnn_class_loss': 1.0,
                    'mrcnn_bbox_loss': 1.0,
                    'mrcnn_mask_loss': 1.0}
    LEARNING_RATE = 0.001   
    MEAN_PIXEL = [248.56, 248.56, 248.56]
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    RPN_NMS_THRESHOLD=0.99  
print('done')

done


Initialize Model and Load Weights:

In [107]:
config = CustomConfig()
#Loading the model in the inference mode
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=path_to_model)
# loading the trained weights o the custom dataset
model.load_weights(path_to_model, by_name=True)

Extract Scar Information From Images:

In [108]:
def extractScarInfoAI(path_to_image, poly_outline):    
    # set up image for model
    new = Image.new('RGB', (512, 512), 0)
    im = cv2.imread(path_to_image, cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (256,559), interpolation = cv2.INTER_LINEAR)
    im = Image.fromarray(im)
    im=im.convert('RGB')
    im=im.resize((234, 512))
    new.paste(im, (139,0))
    new_arr = np.asarray(new)

    # run model
    results = model.detect([new_arr], verbose=1)
    r = results[0]

    # resize image back to original
    new_im = new_arr[0:512,139:373]
    new_im = cv2.resize(new_im, (256,559), interpolation = cv2.INTER_LINEAR)

    polys = []
    classes = []    
    for i in range(0, r['masks'].shape[2]):
        img_gray = r['masks'][:,:,i]      
        mask = img_gray[0:512,139:373]       
        mask = mask.astype(np.uint8)  #convert to an unsigned byte
        mask*=255
        mask = cv2.resize(mask, (256,559), interpolation = cv2.INTER_LINEAR)
        contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
        if contours[0] != []:            
            if len(contours[0][0]) < 5:
                contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
            area = cv2.contourArea(contours[0][0])             
            pts = contours[0][0]
            poly_obj = Polygon(pts.reshape((pts.shape[0], 2)))            
            if len(contours[0]) != 0 and area > 5 and poly_outline.contains(poly_obj) == True:   
                polys.append(contours[0])
                classes.append(r['class_ids'][i])    
    if len(polys) != r['masks'].shape[2]:
        print("Warning, more polygons then mask")    
    annots_json = {}
    annots_json['description'] = ''
    annots_json['tags'] = []
    annots_json['size'] = {'height': new_im.shape[0], 'width': new_im.shape[1]}
    objects = []
    for i in range(0,len(polys)):
        f = {}
        f['id'] = i+1
        f['classId'] = 2876852
        f['description'] = ''
        f['geometryType'] = 'polygon'
        f['labelerLogin'] = 'natewag10'
        f['createdAt'] = time.asctime()
        f['updatedAt'] = time.asctime()
        f['tags'] = []
        if classes[i] == 1:
            f['classTitle'] = 'Scar'
        elif classes[i] == 2:
            f['classTitle'] = 'Mute'
        pt = []        
        for p in polys[i][0]:
            pt.append(list(map(int, p[0])))                
        f['points'] = {'exterior': pt, 'interior': []}

        num_contours = 0
        bb_info = []   
        contour = np.array(pt)
        x, y, w, h = cv2.boundingRect(contour)                          
        area = cv2.contourArea(contour)        
        if area > 5:
            (x,y), (ma,MA), angle = cv2.fitEllipseDirect(contour)   
            if MA == 0:
                MA = 1
            aspect_ratio = float(ma)/MA                  
        f['orientation'] = round(angle,2)
        f['length'] = round(MA,2)
        f['width'] = round(ma,2)
        f['aspect'] = round(aspect_ratio,2)
        objects.append(f)   
    annots_json['num_scars'] = len(polys)    
    annots_json['objects'] = objects  
    if 2 in classes:
        annots_json['has_mute'] = 1
    else:
        annots_json['has_mute'] = 0
    return [os.path.basename(path_to_image), polys, classes, new_im, annots_json]


In [ ]:
import time
start_time = time.time()

f = open(path_to_outline_poly)
data = json.load(f)
pts = data['objects'][0]['points']['exterior']
poly_outline = Polygon(np.array(pts))

all_ims = os.listdir(path_to_all_images)
failed = []
completed = []
for j in all_ims:
    if j[-1] == 'g' or j[-1] == 'G':
        try:
            path_to_image, polys, classes, new_im, annots_json = extractScarInfoAI(path_to_all_images + j, poly_outline)
            with open(path_to_save_annotations + j + '.json', 'w') as json_file:
                json.dump(annots_json, json_file) 
            completed.append(j)          
        except:
            print("FAILED: " + j) 
            failed.append(j)  
print ("Completed in: ", round((time.time() - start_time)/60, 2), ' minutes.')

Check if Any Failed

In [110]:
print(failed)
print(len(completed))

[]
2640
